In [12]:
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [2]:
api_key = 'api_key'

In [3]:
def get_geocode_data(address, api_key):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address':address,
        'key':api_key
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status() # Raise an exception for HTTP Headers
        data = response.json()
        return data['results']
    except request.exception.RequestException as e:
        return f'An error ocurres: {e}'
    except KeyError:
        return 'Unable to parse the response'

In [22]:
def get_coords_from_geocode_data(address, api_key):
    df = get_geocode_data(address, api_key)
    latitude = df[0]['geometry']['location']['lat']
    longitude = df[0]['geometry']['location']['lng']
    return latitude, longitude

In [46]:
def distance(address_1, address_2, api_key):
    lat_1, lon_1 = get_coords_from_geocode_data(address_1, api_key)
    lat_2, lon_2 = get_coords_from_geocode_data(address_2, api_key)
    pnt1 = Point(lat_1, lon_1)
    pnt2 = Point(lat_2, lon_2)
    points_df = gpd.GeoDataFrame({'geometry': [pnt1, pnt2]}, crs='EPSG:2180')
    points_df = points_df.to_crs('EPSG:2180')
    points_df2 = points_df.shift() #We shift the dataframe by 1 to align pnt1 with pnt2
    return points_df.distance(points_df2)

In [51]:
def distance(address_1, address_2, api_key): #z chatuGPT, do sprawdzenia kodowanie EPSG
    lat_1, lon_1 = get_coords_from_geocode_data(address_1, api_key)
    lat_2, lon_2 = get_coords_from_geocode_data(address_2, api_key)
    pnt1 = Point(lon_1, lat_1)
    pnt2 = Point(lon_2, lat_2)
    df1 = gpd.GeoDataFrame({'geometry': [pnt1]}, crs='EPSG:4326')
    df2 = gpd.GeoDataFrame({'geometry': [pnt2]}, crs='EPSG:4326')
    df1 = df1.to_crs('EPSG:2180')
    df2 = df2.to_crs('EPSG:2180')
    distance_meters = df1.distance(df2).values[0]
    return distance_meters

In [53]:
add_1 = 'LX Postępu 21C'
add_2 = 'LX Domaniewska 52'
print(f'Współrzędne {add_1}: {round(get_coords_from_geocode_data(add_1, api_key)[0],2)}N, {round(get_coords_from_geocode_data(add_1, api_key)[1],2)}E')
print(f'Współrzędne {add_2}: {round(get_coords_from_geocode_data(add_2, api_key)[0],2)}N, {round(get_coords_from_geocode_data(add_1, api_key)[1],2)}E')
print(f"Odległość: {round(distance(add_1, add_2, api_key)/1000,3)}km")

Współrzędne LX Postępu 21C: 52.18N, 21.0E
Współrzędne LX Domaniewska 52: 52.18N, 21.0E
Odległość: 0.406km
